In [15]:
# Load necessary libraries
# !pip install --upgrade openai
# !pip install langchain
# !pip install langchain_community
# !pip install pymupdf
# !pip install -qU langchain-text-splitters
# !pip install langchain-chroma


In [66]:
OPENAI_API_KEY= YOUR KEY

In [78]:
# THIS IS LANGCHAIN EXTRACTIOIN OF OPEN AI MODEL
from langchain.chat_models import ChatOpenAI
# Langchain consits of chatprompttemplate, prompttemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser

In [166]:
book1_loader = PyMuPDFLoader("/Users/nityaakalra/Desktop/LANGCHAIN/books.pdf")
book2_loader=PyMuPDFLoader("/Users/nityaakalra/Desktop/LANGCHAIN/book2.pdf")

In [167]:
book1_documents= book1_loader.load()
book2_documents= book2_loader.load()

In [168]:
# Splitting the text
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )

In [169]:
# Split the book text into chunks
book1_chunks = text_splitter.split_documents(book1_documents)

# Split the script text into chunks
book2_chunks = text_splitter.split_documents(book2_documents)

In [170]:
for chunk in book1_chunks:
    chunk.metadata['source'] = 'book1'

for chunk in book2_chunks:
    chunk.metadata['source'] = 'book2'

In [171]:
# Combine the chunks
all_chunks = book1_chunks+ book2_chunks

In [172]:
len(all_chunks)

1407

In [173]:
# Now storing them all in a vector DB Chroma

# Creating an openai embedding
embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [174]:
CHROMA_PATH = 'docs/chroma/'
DATA_PATH = "docs/data"

In [175]:
import os
# Define persist directory
persist_directory = CHROMA_PATH

# Ensure the persist directory exists
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory, exist_ok=True)

In [176]:

chroma_db = Chroma.from_documents(documents=all_chunks, embedding=embedding)

In [177]:
# Retreival and Generation Functions
def retrieve_relevant_chunks(query, top_k=5):
    results = chroma_db.similarity_search(query, k=top_k)
    return results

def generate_response(query):
    # Retrieve relevant chunks
    relevant_chunks = retrieve_relevant_chunks(query)
    
    # Prepare context from the retrieved chunks
    context_text = "\n\n".join([f"Source: {chunk.metadata['source']}\n{chunk.page_content}" for chunk in relevant_chunks])
    
    PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context only: {question}
"""

   
    # Create the prompt using the template
    parser = StrOutputParser()
    # Initialize the OpenAI model
    chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model= "gpt-3.5-turbo")
    prompt_template = PromptTemplate.from_template(PROMPT_TEMPLATE)
    chain= prompt_template| chat|parser
    response=chain.invoke({'context': context_text, 'question': query})
    return response
    
   
    

  

In [194]:
query = "Who are the characcters in book 2 and book 1"
response = generate_response(query)
print(response)


The characters in book 2 are a mother, her two daughters, and the princess in the fairy tale written by the eldest daughter. There is no specific mention of characters in book 1 in the provided context.


In [193]:
query = "Who are the characcters in book 2 and book 1 a man called ove"
retrieve_relevant_chunks(query, top_k=5)

[Document(metadata={'author': 'Nityaa Kalra', 'creationDate': "D:20240719090918Z00'00'", 'creator': 'Safari', 'file_path': '/Users/nityaakalra/Desktop/LANGCHAIN/books.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20240719090918Z00'00'", 'page': 105, 'producer': 'macOS Version 14.5 (Build 23F79) Quartz PDFContext', 'source': 'book', 'subject': '', 'title': 'NLP — https:/therustyquill.wordpress.com/wp-content/uploads/2021/05/man-called-ove_-a-novel-a-fredrik-backman', 'total_pages': 286, 'trapped': ''}, page_content='Ove looks at the book, very unimpressed.\n“What kind of sh—nonsense is this anyway? Some talking train? Is there\nnothing about cars?”\n“Maybe there’s something about nutty old men instead,” mutters the\nseven-year-old.\n“I’m not an ‘old man,’” Ove hisses.\n“Clauwn!” the three-year-old cries out jubilantly.\n“And I’m not a CLOWN either!” he roars.\nThe older one rolls her eyes at Ove, not unlike the way her mother often\nrolls her eyes at Ove.\n“She doesn’t mean y

In [107]:
print(all_chunks)

[Document(metadata={'source': 'book', 'file_path': '/Users/nityaakalra/Desktop/LANGCHAIN/books.pdf', 'page': 2, 'total_pages': 286, 'format': 'PDF 1.4', 'title': 'NLP — https:/therustyquill.wordpress.com/wp-content/uploads/2021/05/man-called-ove_-a-novel-a-fredrik-backman', 'author': 'Nityaa Kalra', 'subject': '', 'keywords': '', 'creator': 'Safari', 'producer': 'macOS Version 14.5 (Build 23F79) Quartz PDFContext', 'creationDate': "D:20240719090918Z00'00'", 'modDate': "D:20240719090918Z00'00'", 'trapped': ''}, page_content='Thank you for downloading this Atria Books\neBook.\nSign up for our newsletter and receive special offers, access to bonus content, and info on the\nlatest new releases and other great eBooks from Atria Books and Simon & Schuster.\nCLICK HERE TO SIGN UP\nor visit us online to sign up at\neBookNews.SimonandSchuster.com'), Document(metadata={'source': 'book', 'file_path': '/Users/nityaakalra/Desktop/LANGCHAIN/books.pdf', 'page': 4, 'total_pages': 286, 'format': 'PDF 1

In [187]:
book2_chunks[90].page_content

'the call operator to ask “Aren’t you a cashless bank? Why would anyone want to\nrob that?” which led London to say “Exactly,” which led the call operator to ask\n“Exactly what?” which led London to snap “What do you mean ‘Exactly\nwhat’?” which led to the call operator hitting back with “You were the one who\nstarted it!” which led London to yell “No, you were the one who…,” after which\nthe conversation quickly deteriorated.) Later it turned out that the police oГcer\nthe bank robber saw in the street wasn’t actually a police oГcer but a traГc\nwarden, and if the bank robber hadn’t been so stressed and had been paying\nattention, that would have been obvious and a diАerent escape strategy might\nhave been possible. Which would have made this a much shorter story.\nBut instead the bank robber rushed through the Йrst available open door,\nwhich led to a stairwell, and then there weren’t exactly many options except to\ngo up the stairs. On the top Мoor one of the apartment doors was wid